In [1]:
# Importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib

# Carregando o conjunto de dados
df = pd.read_csv('teste_indicium_precificacao.csv')

# --- Análise Exploratória de Dados (EDA) ---

# Visualizando as primeiras linhas do conjunto de dados
print(df.head())

# Resumo estatístico
print(df.describe())

# Distribuição de preços
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=30, kde=True)
plt.title('Distribuição de Preços')
plt.xlabel('Preço')
plt.ylabel('Frequência')
plt.show()

# Correlações entre variáveis
correlation_matrix = df.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()


# Identificando bairros com maior potencial de retorno de investimento
avg_price_by_neighbourhood = df.groupby('bairro_group')['price'].mean().sort_values(ascending=False)
print("Bairros com Maior Potencial de Retorno de Investimento:")
print(avg_price_by_neighbourhood.head())

# Analisando a influência de número mínimo de noites e disponibilidade no preço
plt.figure(figsize=(14, 6))
sns.scatterplot(data=df, x='minimo_noites', y='price', hue='disponibilidade_365', palette='viridis', size='disponibilidade_365')
plt.title('Influência de Mínimo de Noites e Disponibilidade no Preço')
plt.xlabel('Mínimo de Noites')
plt.ylabel('Preço')
plt.show()

# Explorando padrões nas variáveis relevantes
colunas_relevantes = ['bairro_group', 'bairro', 'room_type', 'minimo_noites', 'disponibilidade_365']

if all(coluna in df.columns for coluna in colunas_relevantes):
    top_location_patterns = df.nlargest(10, 'price')[colunas_relevantes]
    print("Top 10 Locais de Maior Valor:")
    print(top_location_patterns)
else:
    print(f"As colunas {', '.join(colunas_relevantes)} não estão presentes no conjunto de dados.")

# --- Modelagem e Avaliação ---

# Tratando valores ausentes
imputer = SimpleImputer(strategy='mean')
features = ['minimo_noites', 'disponibilidade_365', 'numero_de_reviews', 'reviews_por_mes']
X = df[features]
y = df['price']
X = imputer.fit_transform(X)

# Divisão do conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando e treinando o modelo de regressão linear
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Avaliando o modelo no conjunto de teste
predictions = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# --- Sugestão de Preço para o Apartamento Específico ---

# Supondo um apartamento com as características fornecidas
apartamento_especifico = {
    'minimo_noites': 1,
    'disponibilidade_365': 355,
    'numero_de_reviews': 45,
    'reviews_por_mes': 0.38
}

# Convertendo para DataFrame e tratando valores ausentes
apartamento_especifico_df = pd.DataFrame([apartamento_especifico])
apartamento_especifico_df = imputer.transform(apartamento_especifico_df)

# Predizendo o preço
preco_sugerido = model.predict(apartamento_especifico_df)
print(f'Sugestão de Preço para o Apartamento Específico: {preco_sugerido[0]}')



# Salvar o modelo
joblib.dump(model, 'modelo_precificacao.pkl')


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


FileNotFoundError: [Errno 2] No such file or directory: '/teste_indicium_precificacao.csv'